# Individual Project
# Formal modelling and statistical analysis of TOR

### Author: Leonidas Reppas
### Supervisor: Gethin Norman

#### Libraries:

In [1]:
import random

#### Relay statistics from the TOR network for 2019 (gathered from Tor metrics):
* Roughly 6000 relays
* 2500 guard-flagged
* 1000 exit-flagged
* 1500 guard&exit-flagged
* 2500 Non-flagged or stable/running but cannot acquire Guard and/or Exit flag
* Users: 2,000,000

In [2]:
# Network setup / Relay creation

# Have different global variables for differently flagged nodes
total_nodes = 0
guard_nodes = 0
exit_nodes = 0
guard_exit_nodes = 0
non_flagged_nodes = 0

class Relays():
    '''
    This class deals with the creation of the relay types
    '''
    
    def create_guard_nodes(self, number):
        global guard_nodes, total_nodes
        guard_nodes += number
        
    def create_exit_nodes(self, number):
        global exit_nodes, total_nodes
        exit_nodes += number
    
    def create_guard_exit_nodes(self, number):
        global guard_exit_nodes, total_nodes
        guard_exit_nodes += number
    
    def create_non_flagged_nodes(self, number):
        global non_flagged_nodes
        non_flagged_nodes += number
        
    def calculate_total_nodes(self):
        global guard_nodes, exit_nodes, guard_exit_nodes, non_flagged_nodes, total_nodes
        total_nodes = guard_nodes + exit_nodes + non_flagged_nodes
        
    def print_nodes(self):
        global guard_nodes, exit_nodes, guard_exit_nodes, non_flagged_nodes, total_nodes
        
        print("""
        Number of guard nodes: {}
        Number of exit nodes: {}
        Number of guard & exit nodes: {}
        Number of non-flagged nodes: {}\n
        Number of total nodes in the network: {}
        """. format(guard_nodes, exit_nodes, guard_exit_nodes, non_flagged_nodes, total_nodes))
            
# Testing
relay = Relays()
relay.create_guard_nodes(2500)
relay.create_exit_nodes(1000)
relay.create_guard_exit_nodes(1500)
relay.create_non_flagged_nodes(2500)
relay.calculate_total_nodes()
relay.print_nodes()


        Number of guard nodes: 2500
        Number of exit nodes: 1000
        Number of guard & exit nodes: 1500
        Number of non-flagged nodes: 2500

        Number of total nodes in the network: 6000
        


#### Path selection algorithm implemented:
 * Node selection based on weights, calculated by node bandwidth.
This was an improvement to the uniformly random path selection. Now a node with 10x higher bandwidth than another node,
has 10x as many circuits and probabilistically 10x as much of the traffic. 

#### Bandwidth weight computation according to the network scarcity conditions:
* G: total bandwidth for guard-flagged nodes = 150Gbit/s
* M: total bandwidth for non-flagged nodes = 50Gbit/s
* E: total bandwidth for exit-flagged nodes = 300Gbit/s
* D: total bandwidth for guard&exit-flagged nodes = 275Gbit/s
* T = G + M + E + D = 775Gbit/s

Network scarcity condition: E >= T/3 but G < T/3 (300>258 but 150<258)
So weight solution is(weight_scale=10,000):
* Wgg = weight_scale
* Wgd = (weight_scale*(D - 2*G + E + M)) / (3*D)
* Wmg = 0
* Wee = (weight_scale*(E + M)) / (2*E)
* Wme = weight_scale - Wee
* Wmd = (weight_scale - Wgd) / 2
* Wed = Wmd

In [3]:
# Defining bandwidth values as global variables (in Gbit/s)
G = 150
M = 50
E = 300
D = 275
T = G + M + E + D
weight_scale = 10000
w_gg = 0
w_gd = 0
w_mg = 0
w_md = 0 
w_me = 0
w_ee = 0
w_ed = 0

In [8]:
# Path Selection Algorithm 
class TorPath():
    '''
    This class deals with the bandwidth weight path 
    selection algorithm for the different user models defined
    '''
    
    def calculate_bandwidth_weights(self):
        global w_gg, w_gd, w_mg, w_ee, w_me, w_md, w_ed
        w_gg = weight_scale
        w_gd = (weight_scale*(D - 2*G + E + M)) / (3*D)
        w_mg = 0
        w_ee = (weight_scale*(E + M)) / (2*E)
        w_me = weight_scale - w_ee
        w_md = (weight_scale - w_gd) / 2
        w_ed = w_md
        
    def print_bandwidth_weights(self):
        global w_gg, w_gd, w_mg, w_ee, w_me, w_md, w_ed
        print("""
        Weights for selecting a guard node:
            guard node: {}
            guard&exit node: {}
        Weights for selecting a middle node:
            guard node: {}
            exit node: {}
            guard&exit node: {}
        Weights for selecting an exit node:
            exit node: {}
            guard&exit: {}
        """.format(w_gg, w_gd, w_mg, w_me, w_md, w_ee, w_ed))
    
    def choose_range_guard(self):
        range_selector = random.randrange(1, 137)
        
        if range_selector == 1:
            range_choice = "very low"
        elif range_selector > 1 and range_selector <= 4:
            range_choice = "low"
        elif range_selector > 4 and range_selector <= 16:
            range_choice = "medium"
        elif range_selector > 16 and range_selector <= 46:
            range_choice = "high"
        elif range_selector > 46 and range_selector <= 136:
            range_choice = "very high"
            
        return range_choice
    
    def choose_range_guard_exit(self):
        range_selector = random.randrange(1, 117)
        
        if range_selector == 1:
            range_choice = "very low"
        elif range_selector > 1 and range_selector <= 4:
            range_choice = "low"
        elif range_selector > 4 and range_selector <= 16:
            range_choice = "medium"
        elif range_selector > 16 and range_selector <= 46:
            range_choice = "high"
        elif range_selector > 46 and range_selector <= 116:
            range_choice = "very high"
            
        return range_choice        
    
    def choose_range_exit(self):
        range_selector = random.randrange(1, 142)
        
        if range_selector == 1:
            range_choice = "very low"
        elif range_selector > 1 and range_selector <= 5:
            range_choice = "low"
        elif range_selector > 5 and range_selector <= 21:
            range_choice = "medium"
        elif range_selector > 21 and range_selector <= 61:
            range_choice = "high"
        elif range_selector > 61 and range_selector <= 141:
            range_choice = "very high"
            
        return range_choice
                
    # define the adversary model
    def deploy_adversary(bandwidth, ratio, nodes):
        # if we have two adversary nodes, deploy one guard and one exit node
        if nodes == 2:
            
            if ratio == 1:
                return (bandwidth/2, bandwidth/2)    
            
    def attack_exit(self, user_model, range_selector, adversary_exit):
        if user_model == "typical":
            user_model_var = 0.92
        elif user_model == "irc":
            user_model_var = 0.8
        elif user_model == "bittorrent":
            user_model_var = 0.3
        
        available_exit_nodes = exit_nodes * user_model_var
        
        if range_selector == "very high" and adversary_exit >= 24:
            compromise = random.randrange(1, (available_exit_nodes*0.1))
            if compromise == 1:
                return True
        elif range_selector == "high" and adversary_exit <= 24 and adversary_exit >= 12:
            compromise = random.randrange(1, (available_exit_nodes*0.25)) 
            if compromise == 1:
                return True
        elif range_selector == "medium" and adversary_exit <= 12 and adversary_exit >= 4.8:
            compromise = random.randrange(1, (available_exit_nodes*0.5)) 
            if compromise == 1:
                return True
        elif range_selector == "low" and adversary_exit <= 4.8 and adversary_exit >= 1.2:
            compromise = random.randrange(1, (available_exit_nodes*0.75)) 
            if compromise == 1:
                return True
        elif range_selector == "very low" and adversary_exit <= 1.2 and adversary_exit >= 0.3:
            compromise = random.randrange(1, (available_exit_nodes*0.9)) 
            if compromise == 1:
                return True
        return False
    
    def attack_guard(self, range_selector, adversary_guard):
        if range_selector == "very high" and adversary_guard >= 18:
            compromise = random.randrange(1, 250) 
            if compromise == 1:
                return True
        elif range_selector == "high" and adversary_exit <= 18 and adversary_exit >= 6:
            compromise = random.randrange(1, 625) 
            if compromise == 1:
                return True
        elif range_selector == "medium" and adversary_exit <= 6 and adversary_exit >= 2.4:
            compromise = random.randrange(1, 1250) 
            if compromise == 1:
                return True
        elif range_selector == "low" and adversary_exit <= 2.4 and adversary_exit >= 0.6:
            compromise = random.randrange(1, 1875) 
            if compromise == 1:
                return True
        elif range_selector == "very low" and adversary_exit <= 0.6 and adversary_exit >= 0.23:
            compromise = random.randrange(1, 2250)
            if compromise == 1:
                return True
        return False

    
    def create_streams_typical(self, number):
        # deploy the adversary nodes
        adversary_guard, adversary_exit = deploy_adversary(bandwidth=12, ratio=1, nodes=2)
        
        # path compromises
        partial_compr = 0
        complete_compr = 0
        
        for i in range(number):
            # reset the compromise flags
            exit_compr = False
            guard_compr = False
            
            # select an exit node
            selector = random.randrange(1, w_ee + w_ed)
            
            if selector <= w_ee:
                exit_choice = "exit"
                range_selector = choose_range_exit()
                
                # relay attack
                exit_compr = attack_exit("typical", range_selector, adversary_exit)
                                    
            else:
                exit_choice = "guard&exit"
                range_selector = choose_range_guard_exit()
                
                # relay attack not applicable
                
            # select a middle node
            selector = random.randrange(1, w_me + w_md)
            
            if selector <= w_me:
                middle_choice = "exit"
                range_selector = choose_range_exit()
                
            else:
                middle_choice = "guard&exit"
                range_selector = choose_range_guard_exit()

            # select a guard node
            selector = random.randrange(1, w_gg + w_gd)
            
            if selector <= w_gg:
                guard_choice = "guard"
                range_selector = choose_range_guard()
                
                # relay attack
                guard_compr = attack_guard(range_selector, adversary_guard)
                
            else:
                guard_choice = "guard&exit"
                range_selector = choose_range_guard_exit()
                
                # relay attack not applicable 
            
            if exit_compr and guard_compr:
                complete_compr += 1
            elif exit_compr or guard_compr:
                partial_compr += 1
            
    
    
    def create_streams_irc(self, number):
        # deploy the adversary nodes
        adversary_guard, adversary_exit = deploy_adversary(bandwidth=12, ratio=1, nodes=2)
        
        # path compromises
        partial_compr = 0
        complete_compr = 0
        
        for i in range(number):
            # reset the compromise flags
            exit_compr = False
            guard_compr = False

            # select an exit node
            selector = random.randrange(1, w_ee + w_ed)
            if selector <= w_ee:
                exit_choice = "exit"
                range_selector = choose_range_exit()

                # relay attack
                exit_compr = attack_exit("irc", range_selector, adversary_exit)

                
            else:
                exit_choice = "guard&exit"
                range_selector = choose_range_guard_exit()


            # select a middle node
            selector = random.randrange(1, w_me + w_md)
            if selector <= w_me:
                middle_choice = "exit"
                range_selector = choose_range_exit()
                
            else:
                middle_choice = "guard&exit"
                range_selector = choose_range_guard_exit()


            # select a guard node
            selector = random.randrange(1, w_gg + w_gd)
            if selector <= w_gg:
                guard_choice = "guard"
                range_selector = choose_range_guard()
                
                # relay attack 
                guard_compr = attack_guard(range_selector, adversary_guard)
            else:
                guard_choice = "guard&exit"
                range_selector = choose_range_guard_exit()

    
    def create_streams_bittorrent(self, number):
        # select an exit node
        selector = random.randrange(1, w_ee + w_ed)
        if selector <= w_ee:
            exit_choice = "exit"
        else:
            exit_choice = "guard&exit"
            
        # select a middle node
        selector = random.randrange(1, w_me + w_md)
        if selector <= w_me:
            middle_choice = "exit"
        else:
            middle_choice = "guard&exit"
        
        # select a guard node
        selector = random.randrange(1, w_gg + w_gd)
        if selector <= w_gg:
            guard_choice = "guard"
        else:
            guard_choice = "guard&exit"

            
            
# Testing 
torpath = TorPath()
torpath.calculate_bandwidth_weights()
torpath.print_bandwidth_weights()


        Weights for selecting a guard node:
            guard node: 10000
            guard&exit node: 3939.3939393939395
        Weights for selecting a middle node:
            guard node: 0
            exit node: 4166.666666666667
            guard&exit node: 3030.30303030303
        Weights for selecting an exit node:
            exit node: 5833.333333333333
            guard&exit: 3030.30303030303
        
